In [1]:
#deal with tensors
import torch
import spacy
#handling text data
from torchtext.legacy import data
import torchtext

In [2]:
#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True


In [3]:
spacy.load("en_core_web_sm")

In [4]:
TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float)
fields = [(None, None), ('text',TEXT),('label', LABEL)]
#loading custom dataset
training_data=data.TabularDataset(path = 'quora1.csv',format = 'csv',fields = fields,skip_header = True)

/home/stefan/.virtualenvs/diplom-code-env/lib/python3.8/site-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [5]:
# with open('quora.csv', 'r') as fi, open('quora1.csv', 'w') as fo:
#     fo.write(next(fi))
#     for line in fi:
#         text = line
#         while text[-2] == ',':
#             text = text[:-2] + text[-1]
#         label = text[-2]
#         if label != '1' and label != '0':
#             print(text)
#         end_quotes = text[-4]
#         if end_quotes != '"':
#             text = text[:-3] + '"' + text[-3:]
#         if text[21] != '"':
#             text = text[:21] + '"' + text[21:]
#         fo.write(text)

In [6]:
import random
train_data, valid_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [7]:
#initialize glove embeddings
TEXT.build_vocab(training_data,min_freq=3,vectors = "glove.6B.100d")
LABEL.build_vocab(training_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))

#Word dictionary
print(TEXT.vocab.stoi)

Size of TEXT vocabulary: 20908
Size of LABEL vocabulary: 2
[('?', 108970), ('the', 56455), ('to', 37393), ('a', 31924), (',', 29505), ('of', 27967), ('in', 27052), ('and', 26551), ('is', 24995), ('Why', 24969)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f5e954cd310>>, {'<unk>': 0, '<pad>': 1, '?': 2, 'the': 3, 'to': 4, 'a': 5, ',': 6, 'of': 7, 'in': 8, 'and': 9, 'is': 10, 'Why': 11, 'do': 12, 'What': 13, 'I': 14, 'are': 15, 'How': 16, 'for': 17, 'that': 18, 'it': 19, 'you': 20, '.': 21, 'Is': 22, 'people': 23, 'with': 24, 'have': 25, 'they': 26, 'be': 27, 'on': 28, 'or': 29, 'can': 30, '"': 31, '-': 32, 'so': 33, 'my': 34, "n't": 35, 'their': 36, "'s": 37, 'as': 38, 'not': 39, 'does': 40, 'from': 41, 'like': 42, 'Do': 43, 'when': 44, 'if': 45, 'an': 46, 'about': 47, ')': 48, 'by': 49, 'get': 50, '(': 51, 'Trump': 52, 'there': 53, 'your': 54, 'would': 55, 'Are': 56, 'women': 57, 'who': 58, 'all': 59, 'did': 60, 'If': 61, 'at': 62, 'will': 6

In [8]:
print(len(training_data))

100000


In [9]:
#check whether cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, valid_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

In [10]:
i = 0
for x in valid_iterator:
    # for word in x.text:
    #     if 'counterfeit' in word:
    #         print(x.text)
    if i % 2000 == 0:
        print(i)
    i += 1
    if i > 3:
        break

0


In [11]:
import torch.nn as nn

class classifier(nn.Module):

    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers,
                 bidirectional, dropout):

        #Constructor
        super().__init__()

        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #lstm layer
        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout,
                           batch_first=True)

        #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        #activation function
        self.act = nn.Sigmoid()

    def forward(self, text, text_lengths):

        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]

        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)

        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]

        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)

        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)

        return outputs

In [12]:
#define hyperparameters
size_of_vocab = len(TEXT.vocab)
embedding_dim = 100
num_hidden_nodes = 32
num_output_nodes = 1
num_layers = 2
bidirection = True
dropout = 0.2

#instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes,num_output_nodes, num_layers,
                   bidirectional = True, dropout = dropout)

In [13]:
#architecture
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

#Initialize the pretrained embedding
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

print(pretrained_embeddings.shape)

classifier(
  (embedding): Embedding(20908, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (act): Sigmoid()
)
The model has 2,150,257 trainable parameters
torch.Size([20908, 100])


In [14]:
import torch.optim as optim

#define optimizer and loss
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()

#define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    rounded_preds = torch.round(preds)

    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc

#push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [15]:
def train(model, iterator, optimizer, criterion):

    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #set the model in training phase
    model.train()

    for batch in iterator:

        #resets the gradients after every batch
        optimizer.zero_grad()

        #retrieve text and no. of words
        text, text_lengths = batch.text

        #convert to 1D tensor
        predictions = model(text, text_lengths).squeeze()

        #compute the loss
        loss = criterion(predictions, batch.label)

        #compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)

        #backpropage the loss and compute the gradients
        loss.backward()

        #update the weights
        optimizer.step()

        #loss and accuracy
        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [16]:
def evaluate(model, iterator, criterion):

    #initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    #deactivating dropout layers
    model.eval()

    #deactivates autograd
    with torch.no_grad():

        for batch in iterator:

            #retrieve text and no. of words
            text, text_lengths = batch.text

            #convert to 1d tensor
            predictions = model(text, text_lengths).squeeze()

            #compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            #keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [17]:
N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    #train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)

    #evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')

    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

	Train Loss: 0.335 | Train Acc: 85.51%
	 Val. Loss: 0.281 |  Val. Acc: 88.96%
	Train Loss: 0.236 | Train Acc: 90.94%
	 Val. Loss: 0.278 |  Val. Acc: 89.15%
	Train Loss: 0.196 | Train Acc: 92.76%
	 Val. Loss: 0.286 |  Val. Acc: 88.99%
	Train Loss: 0.161 | Train Acc: 94.16%
	 Val. Loss: 0.322 |  Val. Acc: 88.31%
	Train Loss: 0.127 | Train Acc: 95.40%
	 Val. Loss: 0.365 |  Val. Acc: 88.09%


In [18]:
torch.save(model.state_dict(), 'saved_weights/deep_classifier_example.pt')

In [19]:

#load weights
path='saved_weights/deep_classifier_example.pt'
model.load_state_dict(torch.load(path));
model.eval();

#inference
import spacy
nlp = spacy.load('en_core_web_sm')

def predict(model, sentence):
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]  #tokenize the sentence
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]          #convert to integer sequence
    length = [len(indexed)]                                    #compute no. of words
    tensor = torch.LongTensor(indexed).to(device)              #convert to tensor
    tensor = tensor.unsqueeze(1).T                             #reshape in form of batch,no. of words
    length_tensor = torch.LongTensor(length)                   #convert to tensor
    prediction = model(tensor, length_tensor)                  #prediction
    return prediction.item()

In [20]:
#make predictions
print(predict(model, "Is there discrimination against black men in the US?"))

#insincere question
print(predict(model, "Why do black men think they should be treated well?"))

0.803727924823761
0.9885077476501465
